In [53]:
from sklearn.externals import joblib

In [54]:
import requests

In [2]:
def pages_fetch(category, tree_depth=0):
    
    output = []
    pages_title_list = []
    pages_id_list    = []
    
    params = {'list'   : 'categorymembers',
              'cmtitle':  category,
              'action' : 'query',
              'format' : 'json',
              'cmlimit': 'max',
              }

    response = requests.get("https://en.wikipedia.org/w/api.php?", params)

    data = response.json()
    data_dict = data['query']['categorymembers']

    pages   = list(filter(lambda x: x['ns'] == 0, data_dict))
    subcats = list(filter(lambda x: x['ns'] == 14, data_dict))

    for i in pages:
        pages_title_list.append(i['title'])
        pages_id_list.append(i['pageid'])
    
    zipped = list(zip(pages_title_list, pages_id_list))
    output.extend(zipped)
        
    if (len(subcats) > 0) and (tree_depth > 0):
        tree_depth -= 1
        for i in subcats:
            new_category = i['title']
            output.extend(pages_fetch(new_category, tree_depth))
            
        return output
    
    else:
        return output

In [3]:
result = pages_fetch('Category:Machine_learning', tree_depth=3)

In [4]:
results_ml = result

In [5]:
len(results_ml)

1624

In [6]:
results_bs = pages_fetch('Category:Business_software', tree_depth=2)

In [7]:
len(results_bs)

4072

In [1]:
def dedupe_list(page_list):
    '''
    Eliminates any repeated values and returns a list of unique values.
    '''
    return list(set(page_list))

In [10]:
ml_pages = dedupe_list(results_ml)

bs_pages = dedupe_list(results_bs)

In [55]:
len(ml_pages)

1108

In [56]:
len(bs_pages)

3017

In [50]:
def get_pages_info(pages_list):
    
    pages_list_of_dicts = []
    
    for name, pageid in pages_list:
        
        page_dict = {}

        params = {'action' : 'parse',
                  'format' : 'json',
                  'page'   :  name,
                  'prop'   : 'text'
                  }

        response = requests.get("https://en.wikipedia.org/w/api.php?", params)
        
        data = response.json()

        raw_hmtl = data['parse']['text']['*']

        clean_text = cleanhtml(raw_hmtl).replace('\n', '')
        
        page_dict['text'] = clean_text
        page_dict['title'] = name
        page_dict['pageid'] = pageid
        
        pages_list_of_dicts.append(page_dict)
                
    return pages_list_of_dicts

In [ ]:
ml_pages_info = get_pages_info(ml_pages)

In [48]:
joblib.dump(ml_pages_info, 'ml_pages_info.pkl')

['ml_pages_info.pkl']

In [ ]:
bs_pages_info = get_pages_info(bs_pages)

In [52]:
joblib.dump(bs_pages_info, 'bs_pages_info.pkl')

['bs_pages_info.pkl']